In [24]:
%load_ext autoreload
%autoreload 2
%cd C:\MAD4AG
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
C:\MAD4AG


In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import geopandas
from scipy.spatial import distance
import pyproj as proj


import warnings

warnings.filterwarnings('ignore')

## filter ppl by
- participating work activity
- county
- urban/rural
- commute distance (is it above or below the median in the survey)
- avg trip distance

In [26]:
file_name = f'./dbs/intermediate/stops_1_new.parquet'


df_clusters = pd.read_parquet(file_name)
df_clusters = df_clusters[df_clusters.holiday_s != 1]
df_clusters = df_clusters[df_clusters.weekday_s == 1]
df_clusters = df_clusters.drop(['holiday_s', 'weekday_s'], axis=1)

df_clusters.drop_duplicates(subset=['uid', 'cluster'], keep='first', inplace=True)

In [27]:
# read home clusters


df_h = pd.read_parquet(f'./dbs/intermediate/home_inference.parquet')
df_h.drop_duplicates(subset='uid', keep='first', inplace=True)

df_h['home_potential'] = 1

In [28]:
# read work clusters, keep only ppl having home locations

df_w = pd.read_parquet(f'./dbs/intermediate/work_inference.parquet')
df_w.drop_duplicates(subset='uid', keep='first', inplace=True)

In [29]:
# keep ppl having home locations

df_clusters = df_clusters[['uid', 'cluster', 'cluster_lat', 'cluster_lng']]

df_clusters = df_clusters[df_clusters.uid.isin(df_h.uid.unique())]

In [30]:
# add home information to clusters

df_clusters = pd.merge(df_clusters, df_h[['uid', 'cluster','home_potential' ]], on=['uid','cluster'], how='left' )

df_clusters.rename(columns={'home_potential':'act_type'}, inplace=True)

df_clusters['act_type'] = df_clusters.act_type.replace(1, 'home')

In [31]:
# add work information to clusters


df_clusters = pd.merge(df_clusters, df_w[['uid', 'cluster','work_potential' ]], on=['uid','cluster'], how='left' )

df_clusters['act_type'][df_clusters.work_potential==1.00000]='work'

df_clusters.drop(['work_potential'], axis=1, inplace=True)

df_clusters['act_type'] = df_clusters['act_type'].fillna('other')


## calculate the average distance between the home and all activity clusters.

In [32]:
# setup your projections
crs_wgs = proj.Proj(init='epsg:4326') # assuming you're using WGS84 geographic
crs_bng = proj.Proj(init='epsg:3006') # use a locally appropriate projected CRS

# then cast your geographic coordinate pair to the projected system
df_clusters['X'], df_clusters['Y']  = proj.transform(crs_wgs, crs_bng, df_clusters.cluster_lng.values, df_clusters.cluster_lat.values)

In [33]:
df_clusters.to_parquet(f'./dbs/intermediate/df_selected_clusters.parquet')

In [34]:
def avg_dist(data):


    avg_dist = distance.cdist(data[['X', 'Y']][data.act_type=='home'], data[['X', 'Y']][data.act_type!='home'], metric='euclidean')

    avg_dist = np.median(avg_dist, axis=1)


    return pd.Series(dict(avg_dist=float(avg_dist)))

In [35]:
tqdm.pandas()
df_clusters_avg_dist = df_clusters.groupby('uid').progress_apply(avg_dist).reset_index()

  0%|          | 0/280996 [00:00<?, ?it/s]

## calculate the commuting distance between the home and all activity clusters.

In [36]:
def com_dist(data):


    dist_data = distance.cdist(data[['X', 'Y']][data.act_type=='home'], data[['X', 'Y']][data.act_type=='work'], metric='euclidean')

    dist_data = np.mean(dist_data, axis=1)

    return pd.Series(dict(com_dist=float(dist_data)))

In [37]:
tqdm.pandas()
df_clusters_com_dist = df_clusters.groupby('uid').progress_apply(com_dist).reset_index()

  0%|          | 0/280996 [00:00<?, ?it/s]

In [41]:
df_act = pd.read_parquet(f'./dbs/intermediate/indi_weights.parquet')

df_act['county']= df_act['Deso'].str[:2]

df_act['urban_density']= df_act['Deso'].str[4]

In [42]:


df_act = pd.merge(df_act, df_clusters_avg_dist, on='uid' )
df_act = pd.merge(df_act,  df_clusters_com_dist, on='uid')

In [43]:
df_act.dropna(subset=['avg_dist'], inplace=True)

In [44]:
# save the table
df_act = df_act[['uid', 'work_potential', 'Deso', 'county', 'urban_density', 'avg_dist',
       'com_dist', 'wt', 'wt_p']]

df_act.rename(columns={'work_potential':"commute"},inplace=True)

In [45]:

df_act.to_parquet(f'./dbs/intermediate/indi_weights.parquet')